# LangChain Tool Calling

Docs:
- [AzureChatOpenAI](https://python.langchain.com/api_reference/openai/chat_models/langchain_openai.chat_models.azure.AzureChatOpenAI.html)
- [Pre-made tools](https://python.langchain.com/docs/integrations/tools/)
- [Custom tools](https://python.langchain.com/docs/how_to/tool_results_pass_to_model/)

See [README.md](/README.md) for how to set up environment.


In [ ]:
import os
from dotenv import load_dotenv

from langchain.tools import tool
import langchain.messages
from pydantic import Field, BaseModel
from langchain_openai import ChatOpenAI

load_dotenv();

# Client & Model
client = ChatOpenAI(
    model="gpt-4o-mini",
    base_url=os.environ["OPENAI_BASE_URL"],
    api_key=lambda: os.environ["OPENAI_API_KEY"]
)


In [ ]:
# Custom tools
@tool
def add(a: int, b: int):
    """Add two integers."""
    return a + b


@tool
def subtract(a: int, b: int):
    """Subtract the second integer from the first integer."""
    return a - b


In [ ]:
# Custom tools with arguments schema
# NOTE: The description helps the LLM generate a value
class MultiplyArgsSchema(BaseModel):
    """Arguments schema for the multiply tool."""

    a: int = Field(description="The first number.")
    b: int = Field(default=5, description="The second number.")


@tool(args_schema=MultiplyArgsSchema)
def multiply(a: int, b: int):
    """Multiply two integers."""
    return a * b


In [ ]:
# Call a tool directly
response = add.invoke({"a": 2, "b": 5})
response


In [ ]:
# Make a tool call + response

# Bind tools to LLM
tools = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
}
client_with_tools = client.bind_tools(tools.values())
messages = [
    langchain.messages.SystemMessage("You are a helpful assistant."),
    langchain.messages.HumanMessage("Add 2 and 6."),
]

# Get tool calls
response = client_with_tools.invoke(messages)
messages.append(response)

# Execute tool calls
for tool_call in response.tool_calls:
    tool = tools[tool_call["name"]]
    tool_msg = tool.invoke(tool_call)
    messages.append(tool_msg)

# Call client one more time to create a response
response = client_with_tools.invoke(messages)
messages.append(response)

# Display output
for msg in messages:
    func_call = msg.additional_kwargs.get("tool_calls", [{}])[-1].get("function", {})
    if func_call:
        tool_content = f"{func_call.get('name')}({func_call.get('arguments')})".replace(
            ":", ": "
        ).replace(",", ", ")
        print(f"tool_call: {tool_content}")
    else:
        print(f"{msg.type}: {msg.content}")


In [ ]:
# Make several tool calls in succession

messages = [
    langchain.messages.SystemMessage("You are a helpful assistant"),
    langchain.messages.HumanMessage(
        "Add 2 and 6 and then multiply the result by 2."
    ),
]

# Loop until the LLM doesn't return any new tool_calls
response = None
while response is None or response.tool_calls:
    response = client_with_tools.invoke(messages)
    messages.append(response)

    for tool_call in response.tool_calls:
        tool = tools[tool_call["name"]]
        tool_msg = tool.invoke(tool_call)
        messages.append(tool_msg)

# Display output
for msg in messages:
    func_call = msg.additional_kwargs.get("tool_calls", [{}])[-1].get("function", {})
    if func_call:
        tool_content = f"{func_call.get('name')}({func_call.get('arguments')})".replace(
            ":", ": "
        ).replace(",", ", ")
        print(f"tool_call: {tool_content}")
    else:
        print(f"{msg.type}: {msg.content}")
